In [1]:
ROOT_FOLDER = '/root/jupyter_home/TF2_Object_Detection'

In [ ]:
cd $ROOT_FOLDER/models/research

In [ ]:
!cp /root/jupyter_home/YOLO_Object_Detection/Fire/fire-dataset.zip .

In [ ]:
!unzip fire-dataset.zip

In [ ]:
fire_class = [
    "fire"
]

with open("fire_map.pbtxt", "w") as f:
    for idx, class_name in enumerate(fire_class, 1):
        f.write("item {\n")
        f.write(f"  id: {idx}\n")
        f.write(f"  name: '{class_name}'\n")
        f.write("}\n\n")

In [ ]:
!mkdir $ROOT_FOLDER/models/research/Firedevkit
!mkdir $ROOT_FOLDER/models/research/Firedevkit/Fire

In [ ]:
!mkdir $ROOT_FOLDER/models/research/Firedevkit/Fire/ImageSets
!mkdir $ROOT_FOLDER/models/research/Firedevkit/Fire/ImageSets/Main
!mkdir $ROOT_FOLDER/models/research/Firedevkit/Fire/Annotations

In [ ]:
cd $ROOT_FOLDER/models/research/Firedevkit/Fire/Annotations

In [ ]:
!cp -r $ROOT_FOLDER/models/research/fire-dataset/train/annotations/*.xml .
!cp -r $ROOT_FOLDER/models/research/fire-dataset/validation/annotations/*.xml .

In [ ]:
!mkdir $ROOT_FOLDER/models/research/Firedevkit/Fire/JPEGImages

In [ ]:
cd $ROOT_FOLDER/models/research/Firedevkit/Fire/JPEGImages

In [ ]:
!cp -r $ROOT_FOLDER/models/research/fire-dataset/train/images/*.jpg .
!cp -r $ROOT_FOLDER/models/research/fire-dataset/validation/images/*.jpg .

In [ ]:
cd $ROOT_FOLDER/models/research/fire-dataset

In [ ]:
import os

folder_path = './train/images'
output_file = 'fire_train.txt'

with open(output_file, 'w') as f:
    for filename in os.listdir(folder_path):
        name, _ = os.path.splitext(filename)
        f.write(name + '\n')

folder_path = './validation/images'
output_file = 'fire_val.txt'

with open(output_file, 'w') as f:
    for filename in os.listdir(folder_path):
        name, _ = os.path.splitext(filename)
        f.write(name + '\n')

In [ ]:
cd $ROOT_FOLDER/models/research/Firedevkit/Fire/ImageSets/Main

In [ ]:
!cp $ROOT_FOLDER/models/research/fire-dataset/fire_train.txt .
!cp $ROOT_FOLDER/models/research/fire-dataset/fire_val.txt .

In [ ]:
cd $ROOT_FOLDER/models/research

In [ ]:
!python ./create_pascal_tf_record.py \
    --data_dir=Firedevkit \
    --year=Fire \
    --set=fire_train \
    --output_path=./fire_train.tfrecord \
    --label_map_path=./fire_map.pbtxt

In [ ]:
!python ./create_pascal_tf_record.py \
    --data_dir=Firedevkit \
    --year=Fire \
    --set=fire_val \
    --output_path=./fire_val.tfrecord \
    --label_map_path=./fire_map.pbtxt

In [ ]:
!mkdir $ROOT_FOLDER/fire_object_detection_model
!mkdir $ROOT_FOLDER/fire_object_detection_model/data
!mkdir $ROOT_FOLDER/fire_object_detection_model/models
!mkdir $ROOT_FOLDER/fire_object_detection_model/models/fire_model

In [ ]:
!cp $ROOT_FOLDER/models/research/fire_map.pbtxt $ROOT_FOLDER/fire_object_detection_model/data
!cp $ROOT_FOLDER/models/research/fire_train.tfrecord $ROOT_FOLDER/fire_object_detection_model/data
!cp $ROOT_FOLDER/models/research/fire_val.tfrecord $ROOT_FOLDER/fire_object_detection_model/data

In [ ]:
!cp -r $ROOT_FOLDER/pretrained_model/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8/checkpoint $ROOT_FOLDER/fire_object_detection_model

In [2]:
cd $ROOT_FOLDER/models/research

/root/jupyter_home/TF2_Object_Detection/models/research


In [ ]:
!python object_detection/model_main_tf2.py \
    --pipeline_config_path=$ROOT_FOLDER/fire_object_detection_model/models/fire_model/my_model.config \
    --model_dir=$ROOT_FOLDER/fire_object_detection_model/models/fire_model \
    --alsologtostderr

In [ ]:
# import

import os
import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as ipython_image

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [5]:
# Utilities

def load_image_into_numpy_array(path):
  # Load an image from file into a numpy array.

  # Args:   
  #  path: the file path to the image

  # Returns:
  #  uint8 numpy array with shape (img_height, img_width, 3)
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)


def get_keypoint_tuples(eval_config):
  # Return a tuple list of keypoint edges from the eval config.
  
  # Args:
  #  eval_config: an eval config containing the keypoint edges
  
  # Returns:
  #  a list of edge tuples, each in the format (start, end)
    tuple_list = []
    kp_list = eval_config.keypoint_edge
    for edge in kp_list:
        tuple_list.append((edge.start, edge.end))
    return tuple_list

In [9]:
# Build a detection model and load pre-trained model weights

pipeline_config = ROOT_FOLDER + '/fire_object_detection_model/models/fire_model/my_model.config'
model_dir = ROOT_FOLDER +'/fire_object_detection_model/models/fire_model'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# # Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)

# ckpt-1의 값은 수정.
ckpt.restore(os.path.join(model_dir, 'ckpt-1')).expect_partial()

def get_model_detection_function(model):
  # Get a tf.function for detection.

    @tf.function
    def detect_fn(image):
    # Detect objects in image.
        image, shapes = model.preprocess(image)
        prediction_dict = model.predict(image, shapes)
        detections = model.postprocess(prediction_dict, shapes)
        
        return detections, prediction_dict, tf.reshape(shapes, [-1])
    
    return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [10]:
# Load label map data (for plotting).

label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
# Visualization

# image_dir = 'models/research/object_detection/test_images/'
# image_path = os.path.join(image_dir, 'image2.jpg')
image_dir = ROOT_FOLDER  + '/test_images'
image_path = os.path.join(image_dir, 'fire.jpg')
image_np = load_image_into_numpy_array(image_path)

# Things to try:
# Flip horizontally
# image_np = np.fliplr(image_np).copy()

# Convert image to grayscale
# image_np = np.tile(
#     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

# # Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in detections:
    keypoints = detections['detection_keypoints'][0].numpy()
    keypoint_scores = detections['detection_keypoint_scores'][0].numpy()

# print(detections['detection_boxes'][0].numpy())
# print(detections['detection_classes'][0].numpy())
# print(detections['detection_scores'][0].numpy())

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      # min_score_thresh를 적절히 설정
      min_score_thresh=.70,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=get_keypoint_tuples(configs['eval_config'])
      )

# print(type(image_np_with_detections), image_np_with_detections.shape)
tmp = Image.fromarray(image_np_with_detections)
tmp.save('./test_result.jpg')
ipython_image('./test_result.jpg')
# plt.figure(figsize=(12,16))
# plt.imshow(image_np_with_detections)
# plt.show()